In [1]:
# import dependicies
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import time
import pandas as pd
import numpy as np

In [2]:
# get url
url = 'https://mars.nasa.gov/news/'

In [3]:
# get html
response = requests.get(url)

In [4]:
# create bs obj
soup = bs(response.text, 'lxml')

In [5]:
content_titles = soup.find_all('div', class_ = 'content_title')

In [6]:
# initalize lists
titles = []
paragraphs = []

In [7]:
for title in content_titles:
    titles.append(title.a.text.strip())

In [8]:
titles

['NASA Invests in Visionary Technology',
 'NASA is Ready to Study the Heart of Mars',
 'NASA Briefing on First Mission to Study Mars Interior',
 "New 'AR' Mobile App Features 3-D NASA Spacecraft",
 'Witness First Mars Launch from West Coast',
 'NASA InSight Mission to Mars Arrives at Launch Site']

In [9]:
content_paragraphs = soup.find_all('div', class_ = 'rollover_description_inner')

In [10]:
for paragraph in content_paragraphs:
    paragraphs.append(paragraph.text.strip())

In [11]:
# grab latest content
latest_title = titles[0]
latest_paragraph = paragraphs[0]

In [12]:
# use splinter to grab image urls
executable_path = {'executable_path': 'chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)
browser.click_link_by_partial_text('FULL IMAGE')

In [13]:
# wait 3 seconds to search for url
time.sleep(3)
html = browser.html
soup = bs(html, 'html.parser')
featured_image_url = soup.find_all('img', class_ = 'fancybox-image')

In [14]:
featured_image_url = 'https://www.jpl.nasa.gov' + featured_image_url[0]['src']

In [15]:
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA18811_ip.jpg'

In [16]:
executable_path = {'executable_path': 'chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)

In [17]:
html = browser.html
soup = bs(html, 'html.parser')
tweets = soup.find_all('div', class_ = 'js-tweet-text-container')

In [18]:
# get the latest tweet
for tweet in tweets:
    mars_weather = tweet.p.text
    break

In [19]:
mars_weather

'Sol 2022 (April 14, 2018), Sunny, high -4C/24F, low -73C/-99F, pressure at 7.19 hPa, daylight 05:27-17:21'

In [20]:
url = 'https://space-facts.com/mars/'
mars_facts = pd.read_html(url)

In [21]:
mars_facts = mars_facts[0]

In [22]:
mars_facts.columns = ['Category', 'Value']

In [23]:
mars_facts.set_index('Category', inplace=True)

In [24]:
mars_facts

,Value
Category,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [25]:
# obtain high resolution image urls for each of Mar's hemispheres
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
response = requests.get(url)
soup = bs(response.text, 'html.parser')
# grab each link on page and put in list
links = soup.find_all('a', class_ = 'itemLink product-item')

In [26]:
urls = []
for link in links:
    urls.append(link['href'])

In [27]:
# set base url
base_url = 'https://astrogeology.usgs.gov'
# initalize lists
image_urls = []
image_titles = []
# loop through list and visit each page
for url in urls:
    response = requests.get(base_url + url)
    soup = bs(response.text, 'html.parser')
    # grab image url
    image_url = soup.find_all('div', class_ = 'downloads')
    image_urls.append(image_url[0].li.a['href'])
    # grab image title
    image_title = soup.find_all('h2', class_ = 'title')
    image_titles.append(image_title[0].text)

In [28]:
image_titles

['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced']

In [29]:
clean_titles = []

In [30]:
for title in image_titles:
    split = title.split('Enhanced')
    clean_titles.append(split[0].strip())

In [31]:
clean_titles

['Cerberus Hemisphere',
 'Schiaparelli Hemisphere',
 'Syrtis Major Hemisphere',
 'Valles Marineris Hemisphere']

In [32]:
hemisphere_image_urls = []

In [33]:
for i in np.arange(len(image_titles)):
    hemisphere_image_urls.append({"Title" : clean_titles[i],
                                 "img_url" : image_urls[i]})

In [35]:
hemisphere_image_urls

[{'Title': 'Cerberus Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'Title': 'Schiaparelli Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'Title': 'Syrtis Major Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'Title': 'Valles Marineris Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]